# Deep Learning
## Assignment 3
Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in 1_notmnist.ipynb.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

* data as a flat matrix,
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.



In [5]:
# stochastic gradient descent training from Assignment 2
batch_size = 128

# Define beta
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
  # Random Seed  
  tf.set_random_seed(42)  

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # L2 Loss Regularization
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 51.065125
Minibatch accuracy: 5.5%
Validation accuracy: 8.6%
Minibatch loss at step 500: 0.930588
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.722718
Minibatch accuracy: 82.0%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.773793
Minibatch accuracy: 82.0%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 0.792115
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 0.896421
Minibatch accuracy: 78.9%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 0.649315
Minibatch accuracy: 82.0%
Validation accuracy: 81.6%
Test accuracy: 88.1%


Achieved a score of 86.1% without Regularization, after inducing Regularization the score improved to 88.1%

## Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [7]:
# stochastic gradient descent training from Assignment 2
batch_size = 16 #Decreasing batch size from 128

# Define beta
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
  # Random Seed  
  tf.set_random_seed(42)  

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # L2 Loss Regularization
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 51.309898
Minibatch accuracy: 0.0%
Validation accuracy: 9.1%
Minibatch loss at step 500: 1.295460
Minibatch accuracy: 75.0%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 3.059481
Minibatch accuracy: 62.5%
Validation accuracy: 70.7%
Minibatch loss at step 1500: 1.343699
Minibatch accuracy: 81.2%
Validation accuracy: 71.2%
Minibatch loss at step 2000: 1.719652
Minibatch accuracy: 68.8%
Validation accuracy: 72.2%
Minibatch loss at step 2500: 1.172488
Minibatch accuracy: 81.2%
Validation accuracy: 76.9%
Minibatch loss at step 3000: 2.197202
Minibatch accuracy: 56.2%
Validation accuracy: 73.5%
Test accuracy: 81.1%


The Accuracy decresed with decrease in batch size
* 128  : Test accuracy: 85.1%
*  64  : Test accuracy: 85.1%
*  32  : Test accuracy: 81.6%
*  16  : Test accuracy: 81.1%

## Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [9]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
  # Random Seed  
  tf.set_random_seed(42)  

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  # Added Dropout
  hidden_drop = tf.nn.dropout(hidden, 0.5)
  logits = tf.matmul(hidden_drop, weights2) + biases2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))

  # L2 Loss Regularization 
  # loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
  loss = tf.reduce_mean(loss + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid, weights2) + biases2)
  
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(hidden_test, weights2) + biases2)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3688.079102
Minibatch accuracy: 7.8%
Validation accuracy: 21.3%
Minibatch loss at step 500: 21.445364
Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Minibatch loss at step 1000: 0.963248
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.844730
Minibatch accuracy: 80.5%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 0.796829
Minibatch accuracy: 83.6%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 0.861987
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%
Minibatch loss at step 3000: 0.717838
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Test accuracy: 89.8%


## Problem 4
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

* global_step = tf.Variable(0)  # count the number of steps taken.
* learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
* optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [11]:
"""
# Adding more hidden layers
"""
batch_size = 1024
num_hidden_nodes_1 = 4096
num_hidden_nodes_2 =  1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():
  
  # Random Seed  
  tf.set_random_seed(42)  

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes_1], stddev=0.001))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes_1]))
  
  weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes_1, num_hidden_nodes_2], stddev=0.001)) 
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes_2]))
    
  weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes_2, num_labels], stddev=0.001)) 
  biases3 = tf.Variable(tf.zeros([num_labels]))  
    
  # Training computation.
  hidden = tf.matmul(tf_train_dataset, weights1) + biases1
  hidden = tf.nn.relu(hidden)
  hidden = tf.nn.dropout(hidden, 0.5) 
   
  hidden = tf.matmul(hidden, weights2) + biases2
  hidden = tf.nn.relu(hidden)
  hidden = tf.nn.dropout(hidden, 0.5)   
  
  logits = tf.matmul(hidden, weights3) + biases3
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

  # L2 Loss Regularization 
  loss = tf.reduce_mean(loss + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)+ tf.nn.l2_loss(weights3)))

  # Optimizer.
  #global_step = tf.Variable(0)
  #starter_learning_rate = 0.00001
  #learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.96, staircase=True)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  hidden_valid_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  hidden_valid_2 = tf.nn.softmax(tf.matmul(hidden_valid_1, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid_2, weights3) + biases3)
  
  hidden_test_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  hidden_test_2 = tf.nn.relu(tf.matmul(hidden_test_1, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(hidden_test_2, weights3) + biases3)

In [12]:
"""
# Adding more hidden layers
"""
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.331281
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.840418
Minibatch accuracy: 81.2%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.797323
Minibatch accuracy: 83.5%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 0.777124
Minibatch accuracy: 82.5%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.736867
Minibatch accuracy: 85.3%
Validation accuracy: 10.0%
Minibatch loss at step 2500: 0.799658
Minibatch accuracy: 83.9%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 0.710185
Minibatch accuracy: 84.7%
Validation accuracy: 10.0%
Test accuracy: 90.8%


In [13]:
"""
# Adding more hidden layers
"""
batch_size = 128
num_hidden_nodes_1 = 1024
num_hidden_nodes_2 =  1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():
  
  # Random Seed  
  tf.set_random_seed(42)  

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes_1], stddev=0.001))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes_1]))
  
  weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes_1, num_hidden_nodes_2], stddev=0.001)) 
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes_2]))
    
  weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes_2, num_labels], stddev=0.001)) 
  biases3 = tf.Variable(tf.zeros([num_labels]))  
    
  # Training computation.
  hidden = tf.matmul(tf_train_dataset, weights1) + biases1
  hidden = tf.nn.relu(hidden)
  hidden = tf.nn.dropout(hidden, 0.5) 
   
  hidden = tf.matmul(hidden, weights2) + biases2
  hidden = tf.nn.relu(hidden)
  hidden = tf.nn.dropout(hidden, 0.5)   
  
  logits = tf.matmul(hidden, weights3) + biases3
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

  # L2 Loss Regularization 
  loss = tf.reduce_mean(loss + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)+ tf.nn.l2_loss(weights3)))

  # Optimizer.
  global_step = tf.Variable(0)
  starter_learning_rate = 0.3
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  hidden_valid_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  hidden_valid_2 = tf.nn.softmax(tf.matmul(hidden_valid_1, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid_2, weights3) + biases3)
  
  hidden_test_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  hidden_test_2 = tf.nn.relu(tf.matmul(hidden_test_1, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(hidden_test_2, weights3) + biases3)

In [14]:
"""
# Adding more hidden layers
"""
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.309791
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 500: 2.304829
Minibatch accuracy: 13.3%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.895822
Minibatch accuracy: 81.2%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 0.876248
Minibatch accuracy: 82.0%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.837091
Minibatch accuracy: 82.8%
Validation accuracy: 10.0%
Minibatch loss at step 2500: 0.903736
Minibatch accuracy: 81.2%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 0.717776
Minibatch accuracy: 84.4%
Validation accuracy: 10.0%
Test accuracy: 90.2%
